<a href="https://colab.research.google.com/github/5koe/BOSTONDATA/blob/main/notebooks/regression/forward_selection_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
!git clone https://github.com/pharringtonp19/business-analytics.git

fatal: destination path 'business-analytics' already exists and is not an empty directory.


### **Import Packages**

In [72]:
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import jax.numpy as jnp
import jax
import seaborn as sb
from functools import partial

### **Specify Dataset**

In [73]:
dataset = 'brookline'

### **Read In Data Set**

In [74]:
if dataset == 'brookline':
  df = pd.read_csv('/content/business-analytics/datasets/brookline.csv')
else:
  df = pd.read_csv('/content/business-analytics/datasets/CASchools.csv')
  df['Students_per_Teacher'] = df['students'] / df['teachers']
df.head()

,price,stNumber,stName,size,beacon,baseFloor,buildingStyle,elevators,rooms,bedrooms,fullBathrooms,halfBathrooms,garage
0,174000,150,PLEASANT ST,1060,0,4,MID-RISE,1,4,2,1,1,1.0
1,337000,7,LEVERETT ST,831,0,1,DECKER,0,4,2,1,0,0.0
2,850000,24,EUSTON ST,2246,0,1,ROW-END,0,10,6,3,0,0.0
3,516000,417,WASHINGTON ST,1574,0,2,LOW-RISE,0,6,3,2,0,0.0
4,145000,150,PLEASANT ST,669,0,4,MID-RISE,1,3,1,1,0,1.0


### **Create Function**

In [75]:
dep_var = 'price'

In [76]:
eligible_vars = df.columns.drop(['stNumber', 'stName', dep_var])
categorical_vars = ['buildingStyle']
eligible_vars

Index(['size', 'beacon', 'baseFloor', 'buildingStyle', 'elevators', 'rooms',
       'bedrooms', 'fullBathrooms', 'halfBathrooms', 'garage'],
      dtype='object')

In [77]:
eligible_vars_rep = {}
for var in eligible_vars:
  if var in categorical_vars:
    eligible_vars_rep[var] = 'C(' + var +')'
  else:
    eligible_vars_rep[var] = var

eligible_vars_rep

{'size': 'size',
 'beacon': 'beacon',
 'baseFloor': 'baseFloor',
 'buildingStyle': 'C(buildingStyle)',
 'elevators': 'elevators',
 'rooms': 'rooms',
 'bedrooms': 'bedrooms',
 'fullBathrooms': 'fullBathrooms',
 'halfBathrooms': 'halfBathrooms',
 'garage': 'garage'}

In [78]:
reg_formula = dep_var + ' ~ '
reg_formula

'price ~ '

In [79]:
def get_rsqr(reg_formula):
  linear_model = smf.ols(reg_formula, data=df)
  results = linear_model.fit()
  return results.rsquared_adj, results.rsquared

get_rsqr('price ~ size')

(0.7488035137772363, 0.7490352448530876)

In [80]:
reg_formulas = list(map(lambda x: reg_formula + x, eligible_vars_rep.values()))

In [81]:
results1 = np.array(list(map(get_rsqr, reg_formulas)))
results1

array([[0.74880351, 0.74903524],
       [0.00213075, 0.0030513 ],
       [0.00879761, 0.009712  ],
       [0.14786679, 0.1549417 ],
       [0.01425743, 0.01516679],
       [0.44192432, 0.44243915],
       [0.43325557, 0.4337784 ],
       [0.47512808, 0.47561228],
       [0.19675122, 0.19749222],
       [0.14420601, 0.14499549]])

In [82]:
np.argmax(results1[:, 0])

0

In [83]:
var_idx = np.argmax(results1[:, 0])
var_name = list(eligible_vars_rep.keys())[var_idx]
print(var_name)
var_name_repr = list(eligible_vars_rep.values())[var_idx]
del eligible_vars_rep[var_name]
reg_formula = reg_formula + ' + ' + var_name_repr

size


### **Specify the dependent variable**

### **Create list of possible Right Hand Side Variables**

### **Create Dictionary of Variable Name and Regression String Format**

### **Create Regression String**

### **For Loop**

In [84]:
for i in range(len(eligible_vars)):

  # Create a List of regression strings (one for each possible new independent variable)

  # Compute the Rsquared and Adjusted-Rsquared for each regression model

  # Extract the adjusted_rsquared_array (first column of results)

  # Find the index with the largest adjusted_rsquared

  # Use the index to select the corresponding indpendent variable with the largest adjusted_rsquared

  # Append the Variable to the Ordered Variable List

  # Update the Regression Formula

  # Update the list of potential independent variables

  # Use the index to retrieve the adjusted_rsquared and r_squared associated with that index

  # Append the values

SyntaxError: incomplete input (<ipython-input-84-024c33f390c7>, line 21)

### **Plot**

In [ ]:
# Create the figure and axes
fig, ax = plt.subplots(figsize=(10, 6))

# Define width for bars and positions for each pair of bars
bar_width = 0.35
x = np.arange(len(ordered_variables))

# Plot adjusted R-squared and R-squared bars side by side
bars_adj = ax.bar(x - bar_width / 2, optimal_adj_rsqr, width=bar_width, label="Adjusted", alpha=0.7)
bars_rs = ax.bar(x + bar_width / 2, optimal_rsqr, width=bar_width, label="Default", alpha=0.5)

# Add labels, title, and angle the x-axis labels
ax.set_title(r"$R^2$", loc='left')
ax.set_xticks(x)
ax.set_xticklabels(ordered_variables, rotation=45, ha='right')
ax.legend(loc='center left', bbox_to_anchor=(.95, 0.9))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Display values above each bar
for bar in bars_adj:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}', ha='center', va='bottom')

for bar in bars_rs:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}', ha='center', va='bottom')

plt.show()